In [9]:
import pandas as pd
import re
df = pd.read_csv("GovernmentProcurementviaGeBIZ.csv")
df.head()

,tender_no,tender_description,agency,award_date,tender_detail_status,supplier_name,awarded_amt
0,ACR000ETT20300002,INVITATION TO TENDER FOR THE PROVISION OF SERV...,Accounting And Corporate Regulatory Authority,10/11/2020,Awarded by Items,DELOITTE & TOUCHE ENTERPRISE RISK SERVICES PTE...,285000.0
1,ACR000ETT20300002,INVITATION TO TENDER FOR THE PROVISION OF SERV...,Accounting And Corporate Regulatory Authority,10/11/2020,Awarded by Items,KPMG SERVICES PTE. LTD.,90000.0
2,ACR000ETT20300003,PROVISION OF AN IT SECURITY CONTROLS AND OPERA...,Accounting And Corporate Regulatory Authority,9/12/2020,Awarded to Suppliers,ERNST & YOUNG ADVISORY PTE. LTD.,182400.0
3,ACR000ETT20300004,"CONCEPTUALIZATION, DESIGN, BUILD, SET-UP OF NE...",Accounting And Corporate Regulatory Authority,9/3/2021,Awarded to Suppliers,D' PERCEPTION SINGAPORE PTE. LTD.,3071056.4
4,ACR000ETT21000001,"DESIGN, DEVELOPMENT, CUSTOMIZATION, DELIVERY, ...",Accounting And Corporate Regulatory Authority,6/9/2021,Awarded to Suppliers,ALPHA ZETTA PTE. LTD.,2321600.0


# Narrow down to life science related agencies

In [10]:
agencies = ["Public Utilities Board", "National Environment Agency", "Health Sciences Authority", "Agency for Science, Technology & Research", "Singapore Food Agency", "National Parks Board", "Ministry of Healthway", "Ministry of Defense", "Government Technology Agency", "Civil Aviation", "Health","Education","Polytechnic"]

pattern = '|'.join([re.escape(a.lower()) for a in agencies])
mask = df['agency'].str.lower().str.contains(pattern)
subset = df[mask]
len(subset)
suppliers = df['supplier_name'].dropna().unique().tolist()
len(suppliers)

6083

In [11]:
competitor_keywords = ['scientific', 'laboratories', 'laboratory', 'bioscience', 'biosciences', 'chemical', 'instrument', 'analytical', 'biotech', 'biological', 'chromatography', 'genomics', 'diagnostic', 'pharma', 'reagent', 'biosystems', 'bios', 'medical', 'biomed', 'gene']

known_competitors = [
    'Merck', 'Sigma Aldrich', 'Avantor', 'VWR', 'LIFE TECHNOLOGIES', 'Agilent', 'PerkinElmer', 'Shimadzu', 'Waters', 'GE Healthcare', 'Bio-Rad', 'Qiagen', 'Illumina', 'Becton Dickinson', 'BD Diagnostics', 'Beckman Coulter', 'Thermo Fisher', 'Thermo Scientific', 'Axil Scientific', 'SPD Scientific', 'ALtec Instruments', 'Eppendorf', 'Sartorius', 'Horiba', 'Zeiss', 'Olympus', 'EXBIO', 'FUJIFILM', 'Hettich', 'Hitachi', 'Roche', 'Abbott', 'Analytik Jena', 'Mettler Toledo', 'Metrohm', 'Kuhner', 'Lonza', 'Promega', 'Takara', 'BioNex', 'Labquip', 'Labchem', 'Lab Science', 'Stryker', 'Medtronic', 'STORZ' 'Philips Healthcare', 'Siemens health', 'Canon Medical Systems', 'Fujifilm Medical Systems', 'Olympus Medical Systems', 'Sakura Finetek', 'Sysmex', 'Roche', 'Abbott', 'Beckman Coulter', 
]


def is_competitor(name):
    name_lower = str(name).lower()
    # special ignore: 'labour' etc but lab->labour might confuse. We'll restrict 'lab ' prefix or ' lab' maybe.
    for kw in competitor_keywords:
        if kw in name_lower:
            return True
    # specified competitor names
    for comp in known_competitors:
        if comp.lower() in name_lower:
            return True
    return False

competitor_suppliers = [name for name in suppliers if is_competitor(name)]
len(competitor_suppliers)


185

In [12]:
df_comp = subset[subset['supplier_name'].apply(is_competitor)]
#df_comp = df[df['supplier_name'].apply(is_competitor)] for all agencies


In [13]:

summary = df_comp.groupby(['supplier_name','agency']).agg(
    num_awards=('tender_no','count'),
    total_value=('awarded_amt','sum')
).reset_index()
summary.head()

,supplier_name,agency,num_awards,total_value
0,1 BISHAN MEDICAL PTE. LTD.,Health Sciences Authority,1,910000.0
1,1 BISHAN MEDICAL PTE. LTD.,Ministry of Education,1,1.0
2,57 MEDICAL PTE. LTD.,Ministry of Health-Ministry Headquarter,1,2360000.0
3,ABBOTT LABORATORIES (SINGAPORE ) PRIVATE LIMITED,Health Sciences Authority,1,9095020.0
4,ABSOLUTE INSTRUMENT SYSTEMS (PTE.) LTD.,National Environment Agency,3,378848.0


In [14]:
import plotly.express as px
import plotly.io as pio
import copy
# Consolidate total_value by summing across all agencies for each supplier
top_50_suppliers = (
    summary.groupby('supplier_name', as_index=False)
    .agg({'total_value': 'sum'})
    .sort_values(by='total_value', ascending=False)
    .head(50)
)


# Create a stacked bar chart grouped by agency
# -----------------------------------------------------
# 2. “Apple” template
# Swap clone() for copy.deepcopy()
apple = copy.deepcopy(pio.templates["simple_white"])

apple.layout.font = dict(
    family="Helvetica Neue, Arial, sans-serif",
    color="#1d1d1f",
    size=14,
)
apple.layout.title = dict(font=dict(size=24, family="Helvetica Neue"))
apple.layout.colorway = ["#007aff"]

# subtle grid
for ax in ("xaxis", "yaxis"):
    getattr(apple.layout, ax).update(
        gridcolor="#e5e5e7",
        zeroline=False,
        showline=False,
    )

pio.templates["apple"] = apple  # register

# -----------------------------------------------------
# 3. Build chart
fig = px.bar(
    top_50_suppliers,
    y="supplier_name",
    x="total_value",
    text="total_value",
    template="apple",
)

# -----------------------------------------------------
# 4. Final polish
fig.update_traces(
    texttemplate="%{text:,.0f}",     # thousands separator
    textposition="outside",
    marker_line_width=0,
)

fig.update_yaxes(
    title=None,
    categoryorder="total ascending",
)

fig.update_layout(
    title=dict(text="Top 50 Suppliers by Spend", x=0.02, xanchor="left"),
    xaxis_title="Total Value",
    bargap=0.18,
    showlegend=False,
    margin=dict(l=140, r=40, t=60, b=40),
    height=1000,
    width=900,
)

fig.show()

# Let's see the breakdown by Agency for each Supplier

In [15]:


# --- 1. Prerequisites -------------------------------------------------
# Make sure these three columns exist and are the right types
summary['supplier_name'] = summary['supplier_name'].astype(str)
summary['agency']        = summary['agency'].fillna('Unknown').astype(str)
summary['total_value']   = pd.to_numeric(summary['total_value'], errors='coerce').fillna(0)

# --- 2. Identify the 50 biggest suppliers (overall) -------------------
supplier_totals = (
    summary.groupby('supplier_name', as_index=False)['total_value']
           .sum()
           .sort_values('total_value', ascending=False)
           .head(50)
)

top_50_names = supplier_totals['supplier_name']

# --- 3. Keep only rows that belong to those suppliers -----------------
df_top50_detail = summary[summary['supplier_name'].isin(top_50_names)].copy()

# Preserve the ranking order for prettier plotting
df_top50_detail['supplier_name'] = pd.Categorical(
    df_top50_detail['supplier_name'],
    categories=supplier_totals['supplier_name'],  # already sorted desc
    ordered=True
)

df_top50_detail['value_m'] = (df_top50_detail['total_value'] / 1e6).round(2).astype(str) + ' M'


# --- 4. Plot: each bar = supplier, segments = agency ------------------
if "apple" not in pio.templates:
    apple = copy.deepcopy(pio.templates["simple_white"])
    apple.layout.font = dict(
        family="Helvetica Neue, Arial, sans-serif",
        color="#1d1d1f",
        size=14
    )
    apple.layout.title = dict(font=dict(size=24, family="Helvetica Neue"))
    # A restrained Apple-style categorical palette
    apple.layout.colorway = [
        "#007aff",  # blue
        "#34c759",  # green
        "#ff9f0a",  # orange
        "#af52de",  # purple
        "#ff375f",  # pink/red
        "#5e5ce6",  # indigo
    ]
    for ax in ("xaxis", "yaxis"):
        getattr(apple.layout, ax).update(
            gridcolor="#e5e5e7",
            zeroline=False,
            showline=False
        )
    pio.templates["apple"] = apple

# ------------------------------------------------------------------
# 1) Build stacked bar chart

matte_palette = [
    "#FF8A3D",  # orange-peach     (≈ 24°)
    "#FBC02D",  # golden yellow   (≈ 43°)
    "#3BB273",  # emerald green   (≈ 148°)
    "#34C6D9",  # cyan-teal       (≈ 187°)
    "#4B71FF",  # royal blue      (≈ 227°)
    "#9B6EF3",  # violet-indigo   (≈ 260°)
    "#FF5E7E",  # pink-magenta    (≈ 348°)
]
fig = px.bar(
    df_top50_detail,
    y="supplier_name",
    x="total_value",
    color="agency",
    text="value_m",
    orientation="h",
    title="Top 50 suppliers – stacked by agency",
    labels={"supplier_name": "Supplier", "total_value": "Total Value"},
    template="apple",
    color_discrete_sequence=matte_palette,   # <-- new line
)

# ------------------------------------------------------------------
# 2) Trace-level tweaks
fig.update_traces(
    texttemplate="%{text}",              # value_m already has units/format
    textposition="outside",
    marker_line_width=0,
)

# ------------------------------------------------------------------
# 3) Layout polish
fig.update_yaxes(
    title=None,
    categoryorder="total ascending"
)

fig.update_layout(
    title=dict(
        text="Top 50 Suppliers – Stacked by Agency",
        x=0.02, xanchor="left"
    ),
    xaxis_title="Total Value",
    bargap=0.18,
    legend_title_text="Agency",
    legend_traceorder="normal",
    legend_yanchor="top",
    legend_y=0.98,
    legend_xanchor="left",
    legend_x=0.78,
    margin=dict(l=160, r=40, t=60, b=40),
    height=1000,
    width=1600
)

fig.show()

In [16]:
import plotly.express as px

# Sort the summary dataframe by total_value in descending order and select the top 50 suppliers
top_50_suppliers = summary.sort_values(by='total_value', ascending=False).head(50)

# Create an interactive bar chart
fig = px.bar(
    top_50_suppliers,
    y='supplier_name',
    x='total_value',
    text='total_value',
    title='Top 50 Suppliers by Total Value',
    labels={'supplier_name': 'Supplier Name', 'total_value': 'Total Value'},
)

# Update layout for better readability
fig.update_layout(
    xaxis_tickangle=-45,
    yaxis_title='Supplier Name',
    xaxis_title='Total Value',
    height=1000,
    width=1600,
)

fig.update_yaxes(categoryorder='total ascending')

# Show the chart
fig.show()